# TODO:

1. accuracy CANNOT be 100% (but we are using SVM??)
1. study performance of models
1. remove bad features (e.g. id)
1. add remaining good features
1. model and train regression models predicting actual return rate
1. thinks how to continuously train models (every month, or something like that), productionize?



In [37]:
import pandas
import numpy as np

import columns
import helpers

In [2]:
files = [
    '../data/LoanStats3a_securev1.csv',
    '../data/LoanStats3b_securev1.csv',
    '../data/LoanStats3c_securev1.csv',
    '../data/LoanStats3d_securev1.csv',
    '../data/LoanStats_securev1_2016Q1.csv',
    '../data/LoanStats_securev1_2016Q2.csv',
    '../data/LoanStats_securev1_2016Q3.csv',
    '../data/LoanStats_securev1_2016Q4.csv',
    '../data/LoanStats_securev1_2017Q1.csv',
    '../data/LoanStats_securev1_2017Q2.csv',
    '../data/LoanStats_securev1_2017Q3.csv',
    '../data/LoanStats_securev1_2017Q4.csv'
]
dataFrames = {}
for filename in files:
    print("Reading " + filename)
    df = pandas.read_csv(filename, dtype=columns.dtypes)
    dataFrames[filename] = df


Reading ../data/LoanStats3a_securev1.csv
Reading ../data/LoanStats3b_securev1.csv
Reading ../data/LoanStats3c_securev1.csv
Reading ../data/LoanStats3d_securev1.csv
Reading ../data/LoanStats_securev1_2016Q1.csv
Reading ../data/LoanStats_securev1_2016Q2.csv
Reading ../data/LoanStats_securev1_2016Q3.csv
Reading ../data/LoanStats_securev1_2016Q4.csv
Reading ../data/LoanStats_securev1_2017Q1.csv
Reading ../data/LoanStats_securev1_2017Q2.csv
Reading ../data/LoanStats_securev1_2017Q3.csv
Reading ../data/LoanStats_securev1_2017Q4.csv


In [3]:
for key in dataFrames:
    print("Parsing " + key)
    df = dataFrames[key]
    df["f_loan_status"] = df.loan_status.apply(columns.parse_loan_status)

Parsing ../data/LoanStats3a_securev1.csv
Parsing ../data/LoanStats3b_securev1.csv
Parsing ../data/LoanStats3c_securev1.csv
Parsing ../data/LoanStats3d_securev1.csv
Parsing ../data/LoanStats_securev1_2016Q1.csv
Parsing ../data/LoanStats_securev1_2016Q2.csv
Parsing ../data/LoanStats_securev1_2016Q3.csv
Parsing ../data/LoanStats_securev1_2016Q4.csv
Parsing ../data/LoanStats_securev1_2017Q1.csv
Parsing ../data/LoanStats_securev1_2017Q2.csv
Parsing ../data/LoanStats_securev1_2017Q3.csv
Parsing ../data/LoanStats_securev1_2017Q4.csv


In [23]:
trimmed = {}
for key in dataFrames:
    print("Trimming " + key)
    df = dataFrames[key]
    trimmed[key] = df.loc[df.f_loan_status.notnull()]


Trimming ../data/LoanStats3a_securev1.csv
Trimming ../data/LoanStats3b_securev1.csv
Trimming ../data/LoanStats3c_securev1.csv
Trimming ../data/LoanStats3d_securev1.csv
Trimming ../data/LoanStats_securev1_2016Q1.csv
Trimming ../data/LoanStats_securev1_2016Q2.csv
Trimming ../data/LoanStats_securev1_2016Q3.csv
Trimming ../data/LoanStats_securev1_2016Q4.csv
Trimming ../data/LoanStats_securev1_2017Q1.csv
Trimming ../data/LoanStats_securev1_2017Q2.csv
Trimming ../data/LoanStats_securev1_2017Q3.csv
Trimming ../data/LoanStats_securev1_2017Q4.csv


In [24]:
for key in dataFrames:
    print(key)
    print(dataFrames[key].shape)
    print(trimmed[key].shape)


../data/LoanStats3a_securev1.csv
(42535, 152)
(42535, 152)
../data/LoanStats3b_securev1.csv
(188181, 152)
(181326, 152)
../data/LoanStats3c_securev1.csv
(235629, 152)
(210765, 152)
../data/LoanStats3d_securev1.csv
(421095, 152)
(245635, 152)
../data/LoanStats_securev1_2016Q1.csv
(133887, 152)
(57531, 152)
../data/LoanStats_securev1_2016Q2.csv
(97854, 152)
(36764, 152)
../data/LoanStats_securev1_2016Q3.csv
(99120, 152)
(34214, 152)
../data/LoanStats_securev1_2016Q4.csv
(103546, 152)
(28809, 152)
../data/LoanStats_securev1_2017Q1.csv
(96779, 152)
(19286, 152)
../data/LoanStats_securev1_2017Q2.csv
(105451, 152)
(13916, 152)
../data/LoanStats_securev1_2017Q3.csv
(122701, 152)
(8986, 152)
../data/LoanStats_securev1_2017Q4.csv
(118648, 152)
(3871, 152)


In [26]:
file = "../data/LoanStats3a_securev1.csv"
print(dataFrames[key].shape)
print(trimmed[key].shape)

df = dataFrames[file]
print(df["f_loan_status"].unique())
print(df["loan_status"].unique())
print(df[~df.f_loan_status.notnull()].loan_status)

(118648, 152)
(3871, 152)
[1 0]
['Fully Paid' 'Charged Off'
 'Does not meet the credit policy. Status:Fully Paid'
 'Does not meet the credit policy. Status:Charged Off']
Series([], Name: loan_status, dtype: object)


In [31]:
def getFinishedLoans(df):
    return df[df.loan_status.isin(['Fully Paid', 'Charged Off', 'Does not meet the credit policy. Status:Fully Paid', 'Does not meet the credit policy. Status:Charged Off'])]
#     return df[df.loan_status == 'Fully Paid'
#               | df.loan_status == 'Charged Off'
#               | df.loan_status == 'Does not meet the credit policy. Status:Fully Paid'
#               | df.loan_status == 'Does not meet the credit policy. Status:Charged Off']

In [32]:
finishedLoans = {}
for key in dataFrames:
    print(key)
    finishedLoans[key] = getFinishedLoans(dataFrames[key])
    print(dataFrames[key].shape)
    print(trimmed[key].shape)
    print(finishedLoans[key].shape)


../data/LoanStats3a_securev1.csv
(42535, 152)
(42535, 152)
(42535, 152)
../data/LoanStats3b_securev1.csv
(188181, 152)
(181326, 152)
(181326, 152)
../data/LoanStats3c_securev1.csv
(235629, 152)
(210765, 152)
(210765, 152)
../data/LoanStats3d_securev1.csv
(421095, 152)
(245635, 152)
(245635, 152)
../data/LoanStats_securev1_2016Q1.csv
(133887, 152)
(57531, 152)
(57531, 152)
../data/LoanStats_securev1_2016Q2.csv
(97854, 152)
(36764, 152)
(36764, 152)
../data/LoanStats_securev1_2016Q3.csv
(99120, 152)
(34214, 152)
(34214, 152)
../data/LoanStats_securev1_2016Q4.csv
(103546, 152)
(28809, 152)
(28809, 152)
../data/LoanStats_securev1_2017Q1.csv
(96779, 152)
(19286, 152)
(19286, 152)
../data/LoanStats_securev1_2017Q2.csv
(105451, 152)
(13916, 152)
(13916, 152)
../data/LoanStats_securev1_2017Q3.csv
(122701, 152)
(8986, 152)
(8986, 152)
../data/LoanStats_securev1_2017Q4.csv
(118648, 152)
(3871, 152)
(3871, 152)


array(['Fully Paid', 'Charged Off',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [24]:
def getFeatures(df):
    numeric_columns = helpers.getNumericColumns(df)
    nonnullable_columns = helpers.getNonNullableColumns(df)
    numeric_nonnullable_columns = list(set(numeric_columns) & set(nonnullable_columns))
    return numeric_nonnullable_columns


In [29]:
s = set(features[0])

In [34]:
# TODO: show to compare equivalent sets?
for t in features:
    print(s & set(t))
    print(set(t) | s)
    break

{'total_pymnt_inv', 'policy_code', 'last_fico_range_high', 'out_prncp_inv', 'installment', 'total_rec_late_fee', 'revol_bal', 'last_fico_range_low', 'funded_amnt', 'total_pymnt', 'recoveries', 'collection_recovery_fee', 'funded_amnt_inv', 'loan_amnt', 'out_prncp', 'total_rec_prncp', 'last_pymnt_amnt', 'total_rec_int', 'fico_range_low', 'id', 'dti', 'fico_range_high'}
{'total_pymnt_inv', 'policy_code', 'last_fico_range_high', 'out_prncp_inv', 'installment', 'total_rec_late_fee', 'revol_bal', 'last_fico_range_low', 'funded_amnt', 'total_pymnt', 'recoveries', 'collection_recovery_fee', 'funded_amnt_inv', 'loan_amnt', 'out_prncp', 'total_rec_prncp', 'last_pymnt_amnt', 'total_rec_int', 'fico_range_low', 'id', 'dti', 'fico_range_high'}
